In [66]:
from graphservice.neoservice import neoconnection
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata

In [67]:
df = pd.read_csv('data/Resume.csv', encoding='utf-8')
df.head(3)

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [68]:
def get_experience_descriptions(text_html):
    soup = BeautifulSoup(text_html)

    experiences_list = soup.find_all('span', class_='jobline')
    # print(experiences_list)
    if experiences_list:
        # print(True)
        experiences_list = [item.text.strip() for item in experiences_list ]
        experiences_list = [unicodedata.normalize("NFKD", item) for item in experiences_list if len(item)>10]
    
    return experiences_list
    

In [70]:
df['experiences'] = df['Resume_html'].apply(get_experience_descriptions)

In [71]:
df = df.explode('experiences')

In [72]:
df.drop(['Resume_str', 'Resume_html'], axis=1, inplace=True)

In [75]:
df.shape

(11472, 3)

In [74]:
df.to_dict(orient='records')

[{'ID': 16852973,
  'Category': 'HR',
  'experiences': "Helps to develop policies, directs and coordinates activities such as employment, compensation, labor relations, benefits, training, and employee services.  Prepares employee separation notices and related documentation  Keeps records of benefits plans participation such as insurance and pension plan, personnel transactions such as hires, promotions, transfers, performance reviews, and terminations, and employee statistics for government reporting.  Advises management in appropriate resolution of employee relations issues.  Administers benefits programs such as life, health, dental, insurance, pension plans, vacation, sick leave, leave of absence, and employee assistance.     Marketing Associate     Designed and created marketing collateral for sales meetings, trade shows and company executives.  Managed the in-house advertising program consisting of print and media collateral pieces.  Assisted in the complete design and launch of